# Implementação do algoritimo completo, sem compromisso com  perfomance

# Imports

In [1]:
import os
import functools
import operator
import math
import random
import sys
import multiprocessing as mp

import pandas as pd
import numpy as np

# Função match

![Match function](./img/match_function.png)

In [2]:
def cluster_matching_function(weight_matrix,
                              cluster_number,
                              element,
                              prototypes,
                              dissimilarity_matrices):
    """
        :params: weight_matrix: numpy array-like 
                    matriz K x P de pesos das matrizes de dissimilaridades por cluster
                cluster_number: int
                    Número do cluster em questão
                element: int
                    Índice do elemento (entre 0 e N-1)
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade

        :return: float

    """

    # Criando aliases compatíveis com as variáveis da fórmula
    k = cluster_number
    D = dissimilarity_matrices
    p = len(D)
    Gk = prototypes[k]
    l = weight_matrix

    dissimilarities_sum = np.array([dj[element, Gk].sum() for dj in D])

    return np.dot(l[k], dissimilarities_sum)

# Função objetivo

![Objetive function](./img/objective_function.png)

In [3]:
def objective_function(clusters_qtd,
                       elements_qtd,
                       adequacy_criterion,
                       m,
                       weight_matrix,
                       prototypes,
                       dissimilarity_matrices):
    """
        :params: clusters_qtd: int
                    Quantidade total de clusters
                elements_qtd: int
                    Quantidade de elementos da base de dados
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                m: int
                    Fator de ponderação do índice de adequação
                weight_matrix:
                     matriz K x P de pesos das matrizes de dissimilaridades por cluster
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade

        :return: float

    """

    u = np.power(adequacy_criterion, m) # Resolvendo a exponeciação de u de uma vez só
    l = weight_matrix
    D = dissimilarity_matrices
    K = clusters_qtd
    G = prototypes
    N = elements_qtd
    match = cluster_matching_function # Criando um alias para reduzir o nome da função de matching
  
    J = np.array([np.array([u[i, k] * match(l, k, i, G, D) for i in range(N)]).sum() 
          for k in range(K)])


    return J.sum()

# Protótipos

![Prototype function](./img/prototype_function.png)

In [4]:
def get_prototypes(elements_qtd,
                   q,
                   m,
                   s,
                   cluster_number,
                   adequacy_criterion,
                   dissimilarity_matrices,
                   weight_matrix):
    """
        :params:
                elements_qtd: int 
                    Quantidade de elementos da base de dados
                q: int
                    Quantidade de elementos protótipos
                m: int
                    Fator de ponderação do índice de adequação
                s: int
                    Fator de ponderação dos pesos das matrizes
                cluster_number: int
                    Quantidade total de clusters
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                weight_matrix: 
                     matriz K x P de pesos das matrizes de dissimilaridades por cluster

        :return: list

    """

    G = []
    k = cluster_number
    D = dissimilarity_matrices
    u = np.power(adequacy_criterion, m)
    l = np.power(weight_matrix, s)
    N = elements_qtd
    p = len(D)
    
    p_sums = np.empty((p, N))
    
    for j in range(p):
        p_sums[j, :] = (D[j] * l[k,j]).sum(axis=1) 
    
    p_sums = p_sums.sum(axis=0)
  
    p_sums_rotten = u[:, k].flatten() * p_sums
    return p_sums_rotten.argsort()[-q:][::-1]

# Protótipos Dummy

![Prototype function](./img/prototype_function.png)

In [5]:
def get_prototypes_dummy2(elements_qtd,
                       q,
                       m,
                       s,
                       cluster_number,
                       adequacy_criterion,
                       dissimilarity_matrices,
                       weight_matrix):
    G = []
    k = cluster_number
    D = dissimilarity_matrices
    u = np.power(adequacy_criterion, m)
    l = np.power(weight_matrix, s)
    N = elements_qtd
    P = len(D)
    
    while (len(G) != q):
        menor_soma = 999999
        menor_indice = None
        
        for h in range(N): 
            if h in G:
                continue
            
            dists_p = np.array([D[j][:, h] * l[k,j] for j in range(P)]) #shape: NxP
            sums_p = dists_p.sum(axis=0)
            soma = np.dot(u[:, k], sums_p)
            
            if soma < menor_soma:
                menor_soma = soma
                menor_indice = h
                 
        G.append(menor_indice)
        
    return G
            

a = get_prototypes(elements_qtd = 3,
                   q=2,
                   m=1.6,
                   s=1,
                   cluster_number=0,
                   adequacy_criterion = np.array([[1,2,3],[3,2,1],[0,1,3]]),
                   dissimilarity_matrices= [np.array([[0, .5, .1],[.3, 0, .1], [1, .2, 0]])],
                   weight_matrix=np.array([[0.8],[0.5]]))

b = get_prototypes_dummy2(elements_qtd = 3,
                   q=2,
                   m=1.6,
                   s=1,
                   cluster_number=0,
                   adequacy_criterion = np.array([[1,2,3],[3,2,1],[0,1,3]]),
                   dissimilarity_matrices= [np.array([[0, .5, .1],[.3, 0, .1], [1, .2, 0]])],
                   weight_matrix=np.array([[0.8],[0.5]]))
print(a, b)

[1 0] [1, 2]


# Matriz de relevâcia

![Funções de peso](./img/vector_weights_function.png)

In [6]:
def compute_relevance_weights(clusters_qtd,
                              dissimilarity_matrices,
                              prototypes,
                              elements_qtd,
                              adequacy_criterion,
                              m):
    """
        :params:
                clusters_qtd: int
                    Quantidade total de clusters
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                elements_qtd: int
                    Quantidade de elementos da base de dados
                adequacy_criterion: numpy array-like
                    Matriz u de tamanho N x K contendo a índice de adequação 
                    de cada elemente a cada cluster
                m: int
                    Fator de ponderação do índice de adequação

        :return: numpy array of shape K x P

    """

    D = dissimilarity_matrices
    P = len(D)
    Gk = prototypes
    K = clusters_qtd
    N = elements_qtd
    u = np.power(adequacy_criterion, m)
    l = np.zeros((K, P))

    def match(element, Dh, G):
        """
            Função auxiliar para cálculo de match entre um elemento 
            qualquer, os protótipos G de um cluster específico e uma matriz 
            de similaridade específica Dh.
        """

        return Dh[element, G].sum()

    for k in range(K):
        # Calculado o somatório do numerador da equação à esquerda da igualdade
        weight_diss_sum1 = np.array([np.array([u[i, k] * match(i, D[h], Gk[k]) for i in range(N)]).sum()
                            for h in range(P)])
        for j in range(P):
     
            # Calculado o somatório do denominador da equação à esquerda da igualdade
            weight_diss_sum2 = np.array([u[i, k] * match(i, D[j], Gk[k])
                                    for i in range(N)]).sum()
            
#             weight_diss_sum2 = weight_diss_sum1[j]

            # Executando a divisão da fração à esquerda da equação
            l[k, j] = math.pow(weight_diss_sum1.prod(), 1/P) / weight_diss_sum2

    return l

# Grau de pertinência

![Fórmula grau de pertinência](./img/membership_degree.png)

In [7]:
def compute_membership_degree(weight_matrix,
                              prototypes,
                              clusters_qtd,
                              dissimilarity_matrices,
                              elements_qtd,
                              m):
    """
        :params: weight_matrix: numpy array-like 
                    matriz K x P de pesos das matrizes de dissimilaridades por cluster
                prototypes: list like
                    Lista de tamanho K dos protótipos de cada cluster
                clusters_qtd: int
                    Quantidade total de clusters
                dissimilarity_matrices: lista de numpy array
                    Lista de matrizes de dissimilaridade
                elements_qtd: int
                    Quantidade de elementos da base de dados
                m: int
                    Fator de ponderação do índice de adequação

        :return: numpy array NxK

    """
        

    K = clusters_qtd
    G = prototypes
    D = dissimilarity_matrices
    l = weight_matrix
    P = len(D)
    N = elements_qtd
    u = np.zeros((N, K))
    
    match = cluster_matching_function # Criando um alias para reduzir o nome da função de matching

    def ratio(element, k, h):
        r = match(l, k, element, G, D) / match(l, h, element, G, D)
        return math.pow(r, 1/(m-1))

    for i in range(N):
        for k in range(K):
            outter_sum = np.array([ratio(i, k, h) for h in range(K)]).sum()
            u[i, k] = 1/outter_sum

    return u

# Carregando Matrizes

In [8]:
def carregar_matrizes(data_path = None):
    data_path = data_path or "./data"
    data_path = os.path.abspath(data_path)
    
    FAC_FILE = os.path.join(data_path, "mfeat-fac-dissimilarity.npy")
    FOU_FILE = os.path.join(data_path, "mfeat-fou-dissimilarity.npy")
    KAR_FILE = os.path.join(data_path, "mfeat-kar-dissimilarity.npy")

    fac_dis = np.load(FAC_FILE)
    fou_dis = np.load(FOU_FILE)
    kar_dis = np.load(KAR_FILE)
    
    return fac_dis, fou_dis, kar_dis

data_path = sys.argv[1] if len(sys.argv) == 2 else None
fac_dis, fou_dis, kar_dis = carregar_matrizes(data_path)

# Algoritmo completo
> Partitioning fuzzy K-medoids clustering algorithms with relevance weight for each dissimilarity matrix estimated locally

* Parametros: $K = 10; m = 1.6; T = 150; \epsilon = 10^{−10};$
* Devemos considerar a iniciarlizar do vetor de pesos como sendo 1, já que usamos a equação 9 (MFCMdd-RWL-P)

In [9]:
def random_prototypes(K, N, q, seed):
    elements = set(range(N))
    protos = []
    random.seed(seed)
    for k in range(K):
        protos.append(random.sample(elements, q))
        elements -= set(protos[-1])

    return protos

def assert_relevance_weights_prod_one(l):
    # l.shape == (K,P)
    prods_p = l.prod(axis=1)
    assert round(prods_p.sum()) == l.shape[0], f"O produto dos pesos de relevância não é igual a 1 ({prods_p.sum()})"

def assert_membership_degree_sum_one(u):
     # u.shape == (N,K)
    sums_k = u.sum(axis=1)
    assert round(sums_k.sum()) == u.shape[0], f"A soma dos pesos de relevância não é igual a 1 ({sums_k.sum()})"

def executar_treinamento(dissimilarity_matrices,
                       elements_qtd,
                       K=10,
                       m=1.6,
                       T=150,
                       epsilon=10e-10,
                       q=2, 
                       seed=13082020):

    D = dissimilarity_matrices
    N = elements_qtd
    P = len(D)

    last_lambda = np.ones((K, P))
    last_prototypes = random_prototypes(K, N, q, seed)
    last_membership_degree = None
    last_cost = None
    
    assert_relevance_weights_prod_one(last_lambda)

#     print("Passo 0")
#     print("Calculando matriz de adequação inicial (u0)")
    u0 = compute_membership_degree(weight_matrix=last_lambda,
                                   prototypes=last_prototypes,
                                   clusters_qtd=K,
                                   dissimilarity_matrices=dissimilarity_matrices,
                                   elements_qtd=N,
                                   m=m)
    
#     assert_membership_degree_sum_one(u0)

#     print("Calculando função de custo inicial (J0)")
    J0 = objective_function(clusters_qtd=K,
                            elements_qtd=N,
                            adequacy_criterion=u0,
                            m=m,
                            weight_matrix=last_lambda,
                            prototypes=last_prototypes,
                            dissimilarity_matrices=dissimilarity_matrices)
    
    last_membership_degree = u0
    last_cost = J0
    
    for t in range(1, T):
#         print(f"Passo {t}/{T}")
        
#         print(">> Calculando protótipos")
        new_prototypes = [get_prototypes_dummy2(elements_qtd=N,
                                         q=q,
                                         m=m,
                                         s=1,
                                         cluster_number=k,
                                         adequacy_criterion=last_membership_degree,
                                         dissimilarity_matrices=D,
                                         weight_matrix=last_lambda) for k in range(K)]
        
        #print("new_prototypes.shape", new_prototypes)
        
#         print(">> Calculando matriz de relevâncias")
        new_lambda = compute_relevance_weights(clusters_qtd=K,
                                               dissimilarity_matrices=D,
                                               prototypes=new_prototypes,
                                               elements_qtd=N,
                                               adequacy_criterion=last_membership_degree,
                                               m=m)
        
#         assert_relevance_weights_prod_one(new_lambda)
    
#         print(">> Calculando grau de pertinência")
        new_degree = compute_membership_degree(weight_matrix=new_lambda,
                                               prototypes=new_prototypes,
                                               clusters_qtd=K,
                                               dissimilarity_matrices=dissimilarity_matrices,
                                               elements_qtd=N,
                                               m=m)
    
        
#         assert_membership_degree_sum_one(new_degree)

#         print(">> Calculando função objetivo")
        new_cost = objective_function(clusters_qtd=K,
                                      elements_qtd=N,
                                      adequacy_criterion=new_degree,
                                      m=m,
                                      weight_matrix=new_lambda,
                                      prototypes=new_prototypes,
                                      dissimilarity_matrices=dissimilarity_matrices)

        last_prototypes = new_prototypes
        last_lambda = new_lambda
        last_membership_degree = new_degree
#         print(">> Cost: ", new_cost)
        
        if abs(last_cost - new_cost) <= epsilon:
            last_cost = new_cost
            break
    
        last_cost = new_cost
        
    data = {
        "cost":last_cost,
        "membership_degree":last_membership_degree,
        "prototypes":last_prototypes,
        "weight_matrix":last_lambda,
        "times": t,
        "q": q,
        "K":K,
        "m":m,
        "seed": seed,
    }
    return data

# Executando 100x

In [21]:
def executar_algoritmo(dissimilarity_matrices, N, times=100, **kwargs):

    best = None
    
    seeds = [18082020 + i for i in range(times)]
    with mp.Pool() as p:
        results = []
        for seed in seeds:
            kwargs["seed"] = seed
            r = p.apply_async(executar_treinamento, (dissimilarity_matrices, N), kwargs)
            results.append(r)
            
        for i, r in enumerate(results):
            data = r.get()
            print(f"Execução {i+1}/{times}")
            print(">> Cost: ", data["cost"] )
            if (not best) or data["cost"] < best["cost"]:
                best = data 
        
            
    return best
# print('>> FAC')
# melhor_resultado_fac = executar_algoritmo([fac_dis], 2000, times=100)
# print('>> FOU')
# melhor_resultado_fou = executar_algoritmo([fou_dis], 2000, times=100)
# print('>> KAR')
# melhor_resultado_kar = executar_algoritmo([kar_dis], 2000,times=100)
print('>> TODAS')
melhor_resultado_todas = executar_algoritmo([fac_dis, fou_dis, kar_dis], 2000, times=1, m=1.8, q=2)


>> TODAS
Execução 1/1
>> Cost:  4208.733319173124


## Exportando resultados

In [22]:
import pickle
import pandas as pd
 
def export_best_result(data, file_name):
    with open(file_name, "wb") as f:
        pickle.dump(data, f)

def export_fuzzy_partitions_to_csv(data, file_name):
    df = pd.DataFrame(data["membership_degree"])
    df.to_csv(file_name, index=False, decimal=',')

    
# export_best_result(melhor_resultado_fac, "data/melhor_resultado_fac.pickle")
# export_best_result(melhor_resultado_fou, "data/melhor_resultado_fou.pickle")
# export_best_result(melhor_resultado_kar, "data/melhor_resultado_kar.pickle")
# export_best_result(melhor_resultado_todas, "data/melhor_resultado_todas.pickle")


# export_fuzzy_partitions_to_csv(melhor_resultado_fac, "data/fuzzy_partitions_fac.csv")
# export_fuzzy_partitions_to_csv(melhor_resultado_fou, "data/fuzzy_partitions_fou.csv")
# export_fuzzy_partitions_to_csv(melhor_resultado_kar, "data/fuzzy_partitions_kar.csv")
export_fuzzy_partitions_to_csv(melhor_resultado_todas, "data/fuzzy_partitions_todas.csv")

## Melhores resultados

In [23]:
# print("Melhor custo pro fac", melhor_resultado_fac["cost"])
# print("Melhor custo pro fou", melhor_resultado_fou["cost"])
# print("Melhor custo pro kar", melhor_resultado_kar["cost"])
print("Melhor custo com as três matrizes", melhor_resultado_todas["cost"])

Melhor custo com as três matrizes 4208.733319173124


## Modified partition coefficient

In [24]:
def calc_partition_coefficient(membership_degree):
    n = membership_degree.shape[0]
    membership_degree = np.power(membership_degree, 2)
    return membership_degree.sum(axis = 1).sum()/n

def calc_modified_partition_coefficient(membership_degree):
    c = membership_degree.shape[1]
    vpc = calc_partition_coefficient(membership_degree)
    return 1 - (c/(c-1))*(1-vpc)

In [25]:
# print('>> FAC (VIEW1)', calc_modified_partition_coefficient(melhor_resultado_fac["membership_degree"]))
# print('>> FOU (VIEW2)', calc_modified_partition_coefficient(melhor_resultado_fou["membership_degree"]))
# print('>> KAR (VIEW3)', calc_modified_partition_coefficient(melhor_resultado_kar["membership_degree"]))
print('>> TODAS (VIEW0)', calc_modified_partition_coefficient(melhor_resultado_todas["membership_degree"]))

>> TODAS (VIEW0) 0.0011137930195328494


## Entropy

In [26]:
def calc_partition_entropy(membership_degree):
    n = membership_degree.shape[0]
    membership_degree = np.log10(membership_degree) * membership_degree
    return -membership_degree.sum(axis = 1).sum()/n


In [18]:
# print('>> FAC (VIEW1)', calc_partition_entropy(melhor_resultado_fac["membership_degree"]))
# print('>> FOU (VIEW2)', calc_partition_entropy(melhor_resultado_fou["membership_degree"]))
# print('>> KAR (VIEW3)', calc_partition_entropy(melhor_resultado_kar["membership_degree"]))
print('>> TODAS (VIEW0)', calc_partition_entropy(melhor_resultado_todas["membership_degree"]))

>> TODAS (VIEW0) 0.9988695754085054


## Partições crisp

In [27]:
from itertools import chain

def get_hard_patitions(membership_degree):
    """
        membership_degree: numpy array of shape N x K
    """
    
    u = membership_degree
    members = []
    K = membership_degree.shape[1]
    
    # Obtendo o índice do grupo em que cada elemento possui maior valor de pertencimento
    element_index_membership = u.argsort(axis=1)[:, -1]
    
    for k in range(K):
        # Para cada grupo k, extrai quais dos elementos possui maior grau de pertencimento para ele
        memb = np.where(element_index_membership == k)[0]
        members.append(memb)
    predicted_classes = [(m, k) for m in memb for (k, memb) in enumerate(members)]
    predicted_classes.sort(key=lambda t: t[0])
    predicted_classes = [m[1] for m in enumerate(members)] 
    
    return members, predicted_classes

def get_instances_class(qtd=10):
    return list(chain(*[[i]*200 for i in range(qtd)]))

classses = get_instances_class(10)

In [20]:
membership_degree = melhor_resultado_todas["membership_degree"]
hard_members, predicted_classes = get_hard_patitions(membership_degree)

for group_number, group in enumerate(hard_members):
    print(group_number, group, len(group))

print("Total de elementos somados:",  sum([len(g) for g in hard_members]))

[[2 5 3 ... 1 0 8]
 [5 2 3 ... 1 0 8]
 [5 2 3 ... 1 0 8]
 ...
 [6 9 1 ... 7 5 2]
 [6 9 1 ... 7 5 2]
 [8 0 1 ... 3 2 5]]
0 [1339 1660 1671] 3
1 [] 0
2 [ 206  210  214  217  223  225  236  239  247  249  254  265  268  276
  278  279  281  286  291  299  306  317  320  323  324  325  330  332
  339  345  348  351  352  354  355  356  359  361  364  371  373  375
  376  389  390  398  400  401  426  433  441  512  542  555  578  600
  601  603  604  605  606  607  608  609  610  611  612  613  614  615
  616  617  618  619  620  621  622  623  624  625  626  627  628  629
  630  631  632  634  635  636  637  638  639  640  641  642  643  644
  645  646  647  649  650  651  652  653  654  655  656  657  658  660
  661  662  663  664  665  666  667  668  669  670  671  672  673  674
  675  676  678  679  680  681  682  683  684  685  686  687  688  690
  691  692  693  694  695  696  697  698  699  700  701  703  704  705
  707  708  709  710  711  712  713  714  715  716  717  718  719  72

## Adjusted Rand Index

In [ ]:
from sklearn.metrics import adjusted_rand_score

fac_crisp, fac_classes = get_hard_patitions(melhor_resultado_fac["membership_degree"])
fou_crisp, fou_classes = get_hard_patitions(melhor_resultado_fou["membership_degree"])
kar_crisp, kar_classes = get_hard_patitions(melhor_resultado_fou["membership_degree"])
todas_crisp, todas_classes = get_hard_patitions(melhor_resultado_todas["membership_degree"])

print('>> FAC (VIEW1)', adjusted_rand_score(melhor_resultado_fac["membership_degree"]))
print('>> FOU (VIEW2)', adjusted_rand_score(melhor_resultado_fou["membership_degree"]))
print('>> KAR (VIEW3)', adjusted_rand_score(melhor_resultado_kar["membership_degree"]))
print('>> TODAS (VIEW0)', adjusted_rand_score(melhor_resultado_todas["membership_degree"]))